In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd

In [2]:
def get_soup_from_url_using_selenium(url):
    from selenium import webdriver
    chromepath = "/Users/smavrovic/Documents/GitHub/Python_Traning/Pydot25/resources/chromedriver"
    driver = webdriver.Chrome(chromepath)
    driver.get(url)
    import time
    time.sleep(7)
    htmldata = driver.page_source
    soup = BeautifulSoup(htmldata)
    return soup

driver_path = "/Users/smavrovic/Documents/GitHub/Python_Traning/Pydot25/resources/chromedriver"

def get_soup(url, headers=None, use_selenium=False, wait_for_howlong=7, scroll_down=0):
    if use_selenium: 
        driver = webdriver.Chrome(driver_path)
        driver.get(url)
        time.sleep(wait_for_howlong)
        if scroll_down:
            for i in range(scroll_down):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(3)
        
        data_html = driver.page_source
        driver.close()
    else:
        resp = requests.get(url, headers=headers)
        data_html = resp.text
    soup = BeautifulSoup(data_html)
    return soup

In [130]:
import re
regexp = re.compile(
    r'^Price_mainPriceContainer[a-zA-Z_0-9]+'
)

# for parent in soup.find_all("div", {'class': regexp }):
def getDenMurphysWine(soup):
    data=[]
    for parent in soup.find_all("div", {'class': "product-content default-content" }):
        price = parent.find("span", {'class': "value" })
        if price:
            price = price.text
        name = parent.find("span", {'class': "title overflow" }).text + "-" + parent.find("span", {'class': "subtitle overflow" }).text
#         print(name)
        star = parent.find("img", {'class': "badge ng-star-inserted" })
        if star:
            star=star.get_attribute_list("src")[0]
            star=star.replace('https://media.danmurphys.com.au/dmo/content/Badges/',"")
            star=star.replace('.png',"")
    #         print(star)
        starP = parent.find("img", {'class': "badge pointer ng-star-inserted" })
        if starP:
            starP=starP.get_attribute_list("src")[0]
            starP=starP.replace('https://media.danmurphys.com.au/dmo/content/Badges/',"")
            starP=starP.replace('.png',"")
    #         print(starP)
    #     if span:
    #         print(span)
        row = [name, price, star, starP]
#         print(row)
        data.append(row)
#     noOfPages = int(soup.find("span", {"class": "page-count"}).text.replace(" of ",""))
#     print("Number of pages is ",noOfPages)
    return data

In [3]:
# soup = get_soup_from_url_using_selenium(url)
# data = []
for i in range(11,32):
    url = f"https://www.danmurphys.com.au/red-wine/country-australia/price-20-999999?page={i}&size=120"
    print(f"Using url {url}")
    soup = get_soup(url, use_selenium=True, scroll_down=1)
    data = data + getDenMurphysWine(soup)
    print(len(data))
df=pd.DataFrame(data)
df.columns = ["Name", "Price", "Score", "Award"]
df

Using url https://www.danmurphys.com.au/red-wine/country-australia/price-20-999999?page=11&size=120


NameError: name 'data' is not defined

In [132]:
df.to_csv("redWinesDenMurphys.csv")

In [126]:
len(data)

539